In [1]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import chromadb 
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core import load_index_from_storage
from llama_index.core import Document

from sklearn.metrics.pairwise import cosine_similarity

from dotenv import load_dotenv, find_dotenv 
import os 
from llama_index.llms.groq import Groq 

import gradio as gr
import pandas as pd 
from pathlib import Path

c:\Users\user\Documents\projetos\python\tcc\projeto\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv("aulas_todas_materias.csv")
documents = []

for _, row in df.iterrows():
    content = f"Aula: {row['data']}\n\n{row['texto']}" 
    metadata = {
        'data': row['data'],
        'bimestre': int(row['bimestre']),
        'materia': row['materia'].lower()
    }
    doc = Document(text=content, metadata=metadata)
    documents.append(doc)

documents



[Document(id_='0ae96e4d-db86-4b50-9f14-d85e4712ac5c', embedding=None, metadata={'data': '2 ABR', 'bimestre': 2, 'materia': 'matematica'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Aula: 2 ABR\n\n2 ABR\r\nOBJETIVOS DE APRENDIZAGEM \r\nDO DC-GOEM - (GO-\r\nEMMAT501A) Compreender o \r\nconceito de função polinomial do 1º\r\ngrau, identificando a relação entre \r\nduas variáveis apresentadas em\r\ntextos de origem socioeconômicas \r\ne/ou de natureza técnico ou\r\ncientífica, entre outros para resolver \r\nsituações problemas do cotidiano.\r\nOBJETOS DE CONHECIMENTO\r\nDO DC-GOEM - Funções\r\npolinomiais do 1º\r\ngrau (função afim,\r\nfunção linear,\r\nfunção constante,\r\nfunção identidade)\r\nAplicação de Atividades via \r\nrecursos digitais\r\nRealização de Atividades via recursos \r\ndigitais', path=None, url=None, mimetype=N

In [4]:
print(documents[0].get_metadata_str())

data: 2 ABR
bimestre: 2
materia: matematica


### Documentos em aprtes menores

In [5]:
node_parser = SentenceSplitter(chunk_size=1000)
nodes = node_parser.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes: 100%|██████████| 295/295 [00:00<00:00, 1408.18it/s]


In [6]:
nodes

[TextNode(id_='4ca72a4c-bd0e-4fb7-bb2c-9fefb023e217', embedding=None, metadata={'data': '2 ABR', 'bimestre': 2, 'materia': 'matematica'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0ae96e4d-db86-4b50-9f14-d85e4712ac5c', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'data': '2 ABR', 'bimestre': 2, 'materia': 'matematica'}, hash='52e657422926a6db2138e265a74b358400755812795c10fe5358023b716fe372')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Aula: 2 ABR\n\n2 ABR\r\nOBJETIVOS DE APRENDIZAGEM \r\nDO DC-GOEM - (GO-\r\nEMMAT501A) Compreender o \r\nconceito de função polinomial do 1º\r\ngrau, identificando a relação entre \r\nduas variáveis apresentadas em\r\ntextos de origem socioeconômicas \r\ne/ou de natureza técnico ou\r\ncientífica, entre outros para resolver \r\nsituações problemas do cotidiano.\r\nOBJETOS DE CONHECIMENTO\r\nDO DC-GOEM - Funções\r\npolinomiais do 1º\r\ngr

### Gerando Embeddings

In [7]:
class ChromaEmbeddingWrapper:
    def __init__(self, model_name):
        self.model = HuggingFaceEmbedding(model_name=model_name)
        self.name = model_name
    
    def __call__(self, texts):
        return self.model.encode(texts, show_progress_bar=False).tolist()

In [8]:
current_file = Path().resolve()
model_folder = current_file / "all-MiniLM-L6-v2"
str(model_folder)

'C:\\Users\\user\\Documents\\projetos\\python\\tcc\\projeto\\all-MiniLM-L6-v2'

In [9]:
chroma_client = chromadb.Client()

In [10]:
try:
    chroma_collection = chroma_client.get_or_create_collection("doccuments_llm")
except Exception as e:
    print(f"Erro ao carregar ou criar embeddings {e}")

### Salvando Embeddings no DB

In [11]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [12]:
embed_model = HuggingFaceEmbedding(model_name=str(model_folder))

In [13]:
index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=embed_model)

### Verificar conteúdos semelhantes entre bimestres

In [15]:
def get_text_for_bimestre(bimestre, materia, query_engine):
    prompt = f"Conteúdo do {bimestre}º bimestre de {materia}"
    response = query_engine.query(prompt)
    if hasattr(response, "response"):
        return response.response
    elif isinstance(response, str):
        return response
    else:
        return None
    
def get_embedding(text, embed_model):
    return embed_model([text])[0]


def check_revision_needed(bimestre_atual, materia, query_engine, embed_model, threshold=0.7):
    if bimestre_atual <= 1:
        return False
    
    texto_atual = get_text_for_bimestre(bimestre_atual, materia, query_engine)
    texto_anterior = get_text_for_bimestre(bimestre_atual - 1, materia, query_engine)

    if not texto_atual or not texto_anterior:
        return False
    
    emb_atual = get_embedding(texto_atual, embed_model)  # usa a função com embed_documents
    emb_ant = get_embedding(texto_anterior, embed_model)
    
    sim = cosine_similarity([emb_atual], [emb_ant])[0][0]
    print(f"Similaridade entre bimestre {bimestre_atual} e {bimestre_atual - 1} para {materia}: {sim:.4f}")
    
    return sim >= threshold


### Recuperação de informações

In [16]:
load_dotenv(find_dotenv())

True

In [17]:
GROQ_API = os.environ.get("GROQ_API")

In [18]:
llm = Groq(
    model="llama3-70b-8192",
    api_key=GROQ_API
)

In [19]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=2)

#### Testando o modelo

In [20]:
aluno = "João da Silva"
nota = 4.5
bimestre = 1
materia = "matematica"

def gerar_plano_estudo(aluno, nota, bimestre, materia, query_engine):
    if nota >= 8:
        return f"O aluno {aluno} teve nota {nota} no {bimestre}º bimestre. Nenhum plano de estudo é necessário."

    #revisao = check_revision_needed(bimestre, materia, query_engine, embed_model)

    #revisao_texto = ""
    #if revisao:
    #    revisao_texto = "Observe que o conteúdo deste bimestre está relacionado ao bimestre anterior. Recomenda-se revisar também os conteúdos do bimestre anterior.\n\n"


    prompt = f"""
    O aluno {aluno} obteve nota {nota} no {bimestre}º bimestre na mataria de {materia}.
    
    Por favor, gere um plano de estudo detalhado **APENAS PARA A MATÉRIA {materia.upper()}**, ignorando informações de outras matérias.

    Nota abaixo de 6 é ruim entre 6 e 7 mediana, acima de 7 boa
    Com base no conteúdo das aulas deste bimestre, gere um plano de estudo com:
    - Conteúdos principais a revisar
    - Dicas de como estudar
    - Sugestão de atividades
    - Gere conteúdos complementares para auxiliar no contéudo prinpal
    Use linguagem simples e voltada para o aluno.
    """
    resp = query_engine.query(prompt)
    return resp

In [ ]:
gerar_plano_estudo(aluno="pedro", nota=6.5, bimestre=3, materia="Matematica", query_engine=query_engine)

Response(response='Olá João!\n\nVamos criar um plano de estudo detalhado para você melhorar sua nota em Matemática.\n\n**Conteúdos principais a revisar:**\n\n1. Sistemas e unidades de medida: leitura e conversão de unidades de grandezas diversas.\n2. Funções: interpretação de gráficos e de expressões algébricas.\n3. Produto cartesiano, relações e funções (geral de funções).\n4. Teoria de conjuntos.\n\n**Dicas de como estudar:**\n\n* Leia atentamente as anotações das aulas e revise os conceitos principais.\n* Faça exercícios relacionados aos conteúdos principais para fixar o conhecimento.\n* Assistir aos vídeos das aulas novamente para reforçar a compreensão.\n* Faça resumos dos conteúdos em seu próprio caderno para ter uma visão geral.\n\n**Sugestão de atividades:**\n\n* Faça uma lista de exercícios sobre sistemas e unidades de medida e resolva-os.\n* Desenhe gráficos de funções e tente interpretá-los.\n* Resolva problemas que envolvam produto cartesiano e relações.\n* Estude a teoria 

In [27]:
import gradio as gr

def interface_plano_com_feedback(aluno, nota, bimestre, materia):
    try:
        plano = gerar_plano_estudo(aluno, nota, bimestre, materia, query_engine)
        return plano
    except Exception as e:
        return f"Erro: {str(e)}"

with gr.Blocks() as demo:
    aluno = gr.Textbox(label="Nome do aluno")
    nota = gr.Number(label="Nota do aluno")
    bimestre = gr.Number(label="Bimestre")
    materia = gr.Textbox(label="Matéria")

    btn_gerar = gr.Button("Gerar Plano de Estudo")
    saida_plano = gr.Textbox(label="Plano de Estudo", lines=10)

    btn_gerar.click(interface_plano_com_feedback, inputs=[aluno, nota, bimestre, materia], outputs=saida_plano)

demo.launch()


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
